# Creating a new planner in nuPlan <a name="introduction"></a>


## Setup

In [1]:
# Useful imports
import os
import hydra
import nest_asyncio
from IPython.core.display import display, HTML
from bokeh.io import output_notebook


/tmp/ipykernel_2455/4095267831.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
nest_asyncio.apply()
output_notebook()
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

Loading BokehJS ...

# Simulating the planner <a name="simulation"></a>

In [3]:
%pwd

'/host_home/Repos/nuplan-devkit/experiments'

In [4]:
%env NUPLAN_DATA_ROOT=/home/ehdykhne/Repos/Datasets/nuplan/dataset
%env NUPLAN_MAPS_ROOT=/home/ehdykhne/Repos/Datasets/nuplan/dataset/maps

env: NUPLAN_DATA_ROOT=/home/ehdykhne/Repos/Datasets/nuplan/dataset
env: NUPLAN_MAPS_ROOT=/home/ehdykhne/Repos/Datasets/nuplan/dataset/maps


## Prepare the simulation config

In [5]:
from tutorials.utils.tutorial_utils import construct_simulation_hydra_paths

# Location of paths with all simulation configs
BASE_CONFIG_PATH = os.path.join(os.getenv('NUPLAN_TUTORIAL_PATH', ''), '../nuplan/planning/script')
simulation_hydra_paths = construct_simulation_hydra_paths(BASE_CONFIG_PATH)

# Create a temporary directory to store the simulation artifacts

DATASET_PARAMS = [
    'scenario_builder=nuplan_mini',  # use nuplan mini database (2.5h of 8 autolabeled logs in Las Vegas)
    'scenario_filter=one_continuous_log',  # simulate only one log
    "scenario_filter.log_names=['2021.06.14.16.48.02_veh-12_04057_04438']",
    'scenario_filter.limit_total_scenarios=1',  # use 1 total scenarios
]
ckpt_dir = '/home/ehdykhne/Repos/nuplan-devkit/experiments/pretrained_checkpoints/pdm_offset_checkpoint.ckpt'
#'/home/sacardoz/checkpoints/urbandriver_checkpoint.ckpt'
#"/home/sacardoz/tutorial_vector_framework/training_simple_vector_experiment/train_default_simple_vector/2023.11.23.09.55.21/best_model/epoch.ckpt"
#"/home/sacardoz/training_raster_experiment/train_default_raster/2023.11.23.07.36.36/best_model/epoch.ckpt"
# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized
hydra.initialize(config_path=simulation_hydra_paths.config_path)

# Compose the configuration
cfg = hydra.compose(config_name=simulation_hydra_paths.config_name, overrides=[
    '+simulation=closed_loop_reactive_agents',
    #'model=pgm_hybrid_model',
    'planner=pdm_hybrid_planner',
    f"planner.pdm_hybrid_planner.checkpoint_path={ckpt_dir}" ,
    #'planner.ml_planner.model_config=${model}',
    #f'planner.ml_planner.checkpoint_path={ckpt_dir}',
    #f'observation=idm_agents_observation',
    #'observation.model_config=${model}',
    #f'observation.checkpoint_path={ckpt_dir}',
    'worker=sequential',
    '+occlusion=true',
    '+occlusion.manager_type=wedge', #options: [range, shadow, wedge]
    "hydra.searchpath=[pkg://tuplan_garage.planning.script.config.common, pkg://tuplan_garage.planning.script.config.simulation, pkg://nuplan.planning.script.config.common, pkg://nuplan.planning.script.experiments]",
    *DATASET_PARAMS,
])

## Launch simulation (within the notebook)

In [6]:
from nuplan.planning.script.run_simulation import build_simulation_runners
from nuplan.common.actor_state.tracked_objects_types import AGENT_TYPES, STATIC_OBJECT_TYPES, TrackedObjectType

# Run the simulation loop (real-time visualization not yet supported, see next section for visualization)
runners, common_builder, cfg = build_simulation_runners(cfg)

Global seed set to 0
INFO:nuplan.planning.script.builders.main_callback_builder:Building MultiMainCallback...
INFO:nuplan.planning.script.builders.main_callback_builder:Building MultiMainCallback: 4...DONE!


2023-12-04 23:36:26,261 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:19}  Building WorkerPool...
2023-12-04 23:36:26,263 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:101}  Worker: Sequential
2023-12-04 23:36:26,263 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:102}  Number of nodes: 1
Number of CPUs per node: 1
Number of GPUs per node: 0
Number of threads across all nodes: 1
2023-12-04 23:36:26,263 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:27}  Building WorkerPool...DONE!
2023-12-04 23:36:26,263 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/script/builders/folder_builder.py:32}  Building experiment folders...
2023-12-04 23:36:26,263 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/script/builders/folder_builder.py:35}  

	Folder where all results are stored: /home/ehdykh

In [7]:
runner = runners[0]

In [8]:
runner.simulation.callback.on_simulation_start(runner.simulation.setup)

# Initialize all simulations
runner._initialize()

while runner.simulation.is_simulation_running():
    # Execute specific callback
    runner.simulation.callback.on_step_start(runner.simulation.setup, runner.planner)

    # Perform step
    planner_input = runner._simulation.get_planner_input()

    # Execute specific callback
    runner._simulation.callback.on_planner_start(runner.simulation.setup, runner.planner)

    # Plan path based on all planner's inputs
    trajectory = runner.planner.compute_trajectory(planner_input)

    # Propagate simulation based on planner trajectory
    runner._simulation.callback.on_planner_end(runner.simulation.setup, runner.planner, trajectory)

    iteration = runner.simulation._time_controller.get_iteration()

    runner.simulation.propagate(trajectory)

    # Execute specific callback
    runner.simulation.callback.on_step_end(runner.simulation.setup, runner.planner, runner.simulation.history.last())

runner.simulation.callback.on_simulation_end(runner.simulation.setup, runner.planner, runner.simulation.history)

elapsed time: 0.028219938278198242
elapsed time: 0.02777719497680664
elapsed time: 0.02846693992614746
elapsed time: 0.0270230770111084
elapsed time: 0.027086973190307617
elapsed time: 0.02704000473022461
elapsed time: 0.02687692642211914
elapsed time: 0.02667236328125
elapsed time: 0.027269363403320312
elapsed time: 0.026731491088867188
elapsed time: 0.02642965316772461
elapsed time: 0.02652263641357422
elapsed time: 0.026285171508789062
elapsed time: 0.026721715927124023
elapsed time: 0.02680516242980957
elapsed time: 0.026640892028808594
elapsed time: 0.02759575843811035
elapsed time: 0.02881908416748047
elapsed time: 0.02743077278137207
elapsed time: 0.02760910987854004
elapsed time: 0.026903390884399414
elapsed time: 0.027498245239257812
elapsed time: 0.02749776840209961
elapsed time: 0.027604103088378906
elapsed time: 0.028120994567871094
elapsed time: 0.028057098388671875
elapsed time: 0.02862381935119629
elapsed time: 0.02895808219909668
elapsed time: 0.02965092658996582
elapse

In [9]:
from tutorials.utils.tutorial_utils import visualize_history
visualize_history(runner.simulation._history, runner.scenario, bokeh_port=5006)

INFO:bokeh.server.server:Starting Bokeh server version 2.4.3 (running on Tornado 6.2)
INFO:bokeh.server.tornado:User authentication hooks NOT provided (default user enabled)


2023-12-04 23:37:42,023 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/nuboard/base/experiment_file_data.py:140}  Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.
2023-12-04 23:37:42,024 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/nuboard/base/experiment_file_data.py:140}  Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.
2023-12-04 23:37:42,026 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/nuboard/base/experiment_file_data.py:140}  Error creating dataset. Could not read schema from 'pretrained_checkpoints/gc_pgp_checkpoint.ckpt'. Is this a 'parquet' file?: Could not open Parquet input source 'pretrained_checkpoints/gc_pgp_checkpoint.ckpt': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.
2023-12-04 23:37:

Rendering a scenario: 100%|██████████| 1/1 [00:00<00:00, 157.64it/s]


2023-12-04 23:37:44,012 INFO {/opt/conda/lib/python3.9/site-packages/tornado/web.py:2271}  200 GET / (10.40.117.44) 2030.70ms
2023-12-04 23:37:44,023 INFO {/home/ehdykhne/Repos/nuplan-devkit/tutorials/utils/tutorial_utils.py:267}  Done rendering!


INFO:tornado.access:200 GET / (10.40.117.44) 2030.70ms


2023-12-04 23:37:44,370 INFO {/opt/conda/lib/python3.9/site-packages/tornado/web.py:2271}  200 GET /static/js/bokeh-gl.min.js?v=e5df31fd9010eacff0aa72d315264604b5e34972ba445acea6fce98080eecf33acf2d2986126360faaa5852813cffa16f6f6f4889923318300f062497c02da4e (10.40.117.44) 271.87ms
2023-12-04 23:37:44,398 INFO {/opt/conda/lib/python3.9/site-packages/tornado/web.py:2271}  200 GET /static/js/bokeh.min.js?v=3c61e952b808bb7e346ce828a565a5f23aaf7708d034fa9d0906403813355d45bb4e8d8b0b23a93f032c76831d4f0221846f28699c7f5147caa62e0d31668314 (10.40.117.44) 299.84ms
2023-12-04 23:37:44,400 INFO {/opt/conda/lib/python3.9/site-packages/tornado/web.py:2271}  200 GET /static/js/bokeh-widgets.min.js?v=8a1ff6f5aa0d967f4998d275803bbb111d928fd9f605ef9e1f30cfd021df0e77224ee3d13f83edb3a942f6e4ccc569ee5dd8951a8aa6cb600602463b90c65a87 (10.40.117.44) 301.22ms
2023-12-04 23:37:44,435 INFO {/opt/conda/lib/python3.9/site-packages/tornado/web.py:2271}  200 GET /static/js/bokeh-tables.min.js?v=ae2903e57cf57f52819fdf4

INFO:tornado.access:200 GET /static/js/bokeh-gl.min.js?v=e5df31fd9010eacff0aa72d315264604b5e34972ba445acea6fce98080eecf33acf2d2986126360faaa5852813cffa16f6f6f4889923318300f062497c02da4e (10.40.117.44) 271.87ms
INFO:tornado.access:200 GET /static/js/bokeh.min.js?v=3c61e952b808bb7e346ce828a565a5f23aaf7708d034fa9d0906403813355d45bb4e8d8b0b23a93f032c76831d4f0221846f28699c7f5147caa62e0d31668314 (10.40.117.44) 299.84ms
INFO:tornado.access:200 GET /static/js/bokeh-widgets.min.js?v=8a1ff6f5aa0d967f4998d275803bbb111d928fd9f605ef9e1f30cfd021df0e77224ee3d13f83edb3a942f6e4ccc569ee5dd8951a8aa6cb600602463b90c65a87 (10.40.117.44) 301.22ms
INFO:tornado.access:200 GET /static/js/bokeh-tables.min.js?v=ae2903e57cf57f52819fdf4d938c648982b51c34f73b6e653a0f3bb3c8ab44f338505931ace43eafc1636e215492e2314acf54c54baffb47813b86b4923a7fe0 (10.40.117.44) 37.66ms
INFO:tornado.access:200 GET /static/js/bokeh-mathjax.min.js?v=176c36fdbcd8fc1019fc828101a2804081a35baf4018d7f2633cd263156b593aa73112f400112b662daa0590138b7

2023-12-04 23:37:45,376 INFO {/opt/conda/lib/python3.9/site-packages/tornado/web.py:2271}  200 GET /static/js/bokeh-mathjax.min.js?v=176c36fdbcd8fc1019fc828101a2804081a35baf4018d7f2633cd263156b593aa73112f400112b662daa0590138b74851bc91f1f2a5fbf5416ee8c876c3e0d0c (10.40.117.44) 135.59ms
2023-12-04 23:37:45,551 INFO {/opt/conda/lib/python3.9/site-packages/tornado/web.py:2271}  101 GET /ws (10.40.117.44) 1.63ms
2023-12-04 23:37:45,664 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/nuboard/base/experiment_file_data.py:140}  Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.
2023-12-04 23:37:45,666 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/nuboard/base/experiment_file_data.py:140}  Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.
2023-12-04 23:37:45,666 INFO {/home/ehdykhn

Rendering a scenario: 100%|██████████| 1/1 [00:00<00:00, 10.45it/s]
INFO:tornado.access:200 GET / (10.40.117.44) 805.62ms


2023-12-04 23:37:46,467 INFO {/opt/conda/lib/python3.9/site-packages/tornado/web.py:2271}  200 GET / (10.40.117.44) 805.62ms
2023-12-04 23:37:46,513 INFO {/home/ehdykhne/Repos/nuplan-devkit/tutorials/utils/tutorial_utils.py:267}  Done rendering!
